In [1]:
from nfl_data_py import import_players, import_weekly_data

# Fetch player stats data
player_stats = import_weekly_data([2022])

# Fetch player metadata
players = import_players()

Downcasting floats.


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Combine player stats and player metadata into a single DataFrame
df = pd.merge(player_stats, players, left_on='player_id', right_on='gsis_id', how='left')

# Sort the dataframe by player_id and week
df = df.sort_values(['player_id', 'week'])

# Create the target variable (next week's passing yards)
df['next_week_passing_yards'] = df.groupby('player_id')['passing_yards'].shift(-1)

# Remove the last week for each player (as we don't have next week's data for it)
df = df.groupby('player_id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)

# Encode categorical features
label_encoder = LabelEncoder()
df['position_x'] = label_encoder.fit_transform(df['position_x'])
df['recent_team'] = label_encoder.fit_transform(df['recent_team'])

# Scale numeric features
scaler = StandardScaler()
df[['passing_yards', 'rushing_yards', 'receiving_yards']] = scaler.fit_transform(df[['passing_yards', 'rushing_yards', 'receiving_yards']])

# Remove rows with NaN values
df = df.dropna(subset=['next_week_passing_yards', 'passing_yards', 'rushing_yards', 'receiving_yards'])

/tmp/ipykernel_22862/3283583966.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('player_id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)


In [3]:
X = df[['position_x', 'recent_team', 'passing_yards', 'rushing_yards', 'receiving_yards']]
y = df['next_week_passing_yards']

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Define the model architecture
model = Sequential()
model.add(Input(shape=(5,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)

2024-08-26 09:03:48.499722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-26 09:03:49.417463: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/100


2024-08-26 09:03:50.320919: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-26 09:03:50.390274: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-26 09:03:50.394007: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

126/126 [==============================] - 3s 3ms/step - loss: 5935.6655 - val_loss: 4026.5369
Epoch 2/100
126/126 [==============================] - 0s 3ms/step - loss: 4688.5513 - val_loss: 2651.9541
Epoch 3/100
126/126 [==============================] - 0s 2ms/step - loss: 2280.9663 - val_loss: 1366.1849
Epoch 4/100
126/126 [==============================] - 0s 3ms/step - loss: 1120.7733 - val_loss: 1278.5441
Epoch 5/100
126/126 [==============================] - 0s 2ms/step - loss: 1021.1524 - val_loss: 1275.9760
Epoch 6/100
126/126 [==============================] - 0s 2ms/step - loss: 1001.8184 - val_loss: 1258.5110
Epoch 7/100
126/126 [==============================] - 0s 3ms/step - loss: 985.4756 - val_loss: 1246.4224
Epoch 8/100
126/126 [==============================] - 0s 3ms/step - loss: 962.6625 - val_loss: 1183.2515
Epoch 9/100
126/126 [==============================] - 0s 3ms/step - loss: 948.0475 - val_loss: 1169.1630
Epoch 10/100
126/126 [==============================

In [5]:
import numpy as np

# Select a player (in this case, the first one in the DataFrame)
player_name = "Patrick Mahomes"  # Replace with the name of the player you're interested in
player_stats = df[df['player_display_name'] == player_name].iloc[0]

# Print player information
print(f"Player: {player_stats['player_name']} (ID: {player_stats['player_id']})")
print(f"Position: {player_stats['position_x']}")
print(f"Team: {player_stats['recent_team']}")
print(f"Current week stats:")
print(f"  Passing yards: {player_stats['passing_yards']}")
print(f"  Rushing yards: {player_stats['rushing_yards']}")
print(f"  Receiving yards: {player_stats['receiving_yards']}")

input_data = np.array([[
    player_stats['position_x'],
    player_stats['recent_team'],
    player_stats['passing_yards'],
    player_stats['rushing_yards'],
    player_stats['receiving_yards']
]])

next_week_passing_yards = None

try:
    prediction = model.predict(input_data)
    
    if prediction.shape == (1, 1):
        next_week_passing_yards = prediction[0][0]
        print(f"\nPredicted next week's passing yards: {next_week_passing_yards}")
    else:
        print(f"\nUnexpected prediction shape: {prediction.shape}")
        print(f"Prediction content: {prediction}")
        
except Exception as e:
    print(f"\nAn error occurred during prediction: {e}")

if next_week_passing_yards is None:
    print("\nFailed to generate a valid prediction.")

Player: P.Mahomes (ID: 00-0033873)
Position: 4
Team: 15
Current week stats:
  Passing yards: 4.503370548490142
  Rushing yards: -0.28853030402544877
  Receiving yards: -0.8017950927505366
1/1 [==============================] - 0s 67ms/step

Predicted next week's passing yards: 273.007568359375
